In [2]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import yaml
import subprocess
import os

import runMops
import MopsAnalysis
import MopsReader
import MopsPlotter
from MopsTracker import MopsTracker
from MopsParameters import MopsParameters

% matplotlib inline

In [3]:
dataDrive = "/Volumes/DataCenter/"
dataLocation = "neosimData/colin/dia_sources_to_mops.dat"
subsampleLocation = "neosimData/colin_2016_03_14/"
runLocation = dataDrive + subsampleLocation

In [16]:
def readDetectionsIntoDataframe(detsFile, header=1):
    return pd.read_csv(detsFile, header=header, names=["diaId", "visitId", "ssmId", "ra", "dec", "mjd", "mag", "snr"], delim_whitespace=True)

In [17]:
dets_df = readDetectionsIntoDataframe(dataDrive + dataLocation)
dets_df

,diaId,visitId,ssmId,ra,dec,mjd,mag,snr
0,84768485325931030,197367,1,189.456717,-5.587153,56398.008951,-7.034141,5
1,84768485325931045,197367,1,189.466819,-5.536871,56398.008951,-7.405883,5
2,84768485325931046,197367,1,189.467385,-5.569577,56398.008951,-6.741032,5
3,84768485325931051,197367,1,189.471613,-5.554848,56398.008951,-7.583233,5
4,84768485325931063,197367,1,189.482410,-5.533385,56398.008951,-8.145744,5
5,84768485325931095,197367,1,189.504929,-5.570673,56398.008951,-6.760771,5
6,84768485325931098,197367,1,189.506098,-5.570450,56398.008951,-6.799934,5
7,84768485325931110,197367,1,189.516017,-5.593709,56398.008951,-6.992247,5
8,84768485325931233,197367,1,189.597998,-5.599326,56398.008951,-6.887967,5
9,84768485325931237,197367,1,189.599544,-5.603819,56398.008951,-7.341438,5


In [19]:
reduced_dets_df = dets_df.drop("diaId", 1)
reduced_dets_df

,visitId,ssmId,ra,dec,mjd,mag,snr
0,197367,1,189.456717,-5.587153,56398.008951,-7.034141,5
1,197367,1,189.466819,-5.536871,56398.008951,-7.405883,5
2,197367,1,189.467385,-5.569577,56398.008951,-6.741032,5
3,197367,1,189.471613,-5.554848,56398.008951,-7.583233,5
4,197367,1,189.482410,-5.533385,56398.008951,-8.145744,5
5,197367,1,189.504929,-5.570673,56398.008951,-6.760771,5
6,197367,1,189.506098,-5.570450,56398.008951,-6.799934,5
7,197367,1,189.516017,-5.593709,56398.008951,-6.992247,5
8,197367,1,189.597998,-5.599326,56398.008951,-6.887967,5
9,197367,1,189.599544,-5.603819,56398.008951,-7.341438,5


In [21]:
reduced_dets_df.to_csv(dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat", sep=" ", header=False)

In [22]:
nightly = dataDrive + "neosimData/colin/nightly/"

! rm -rf {nightly}
! mkdir {nightly}

In [23]:
call = ["python", os.getenv("MOPS_DIR") + "/bin/splitByNight.py", "-n", nightly, dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat"]
subprocess.call(call);

In [24]:
parameters = MopsParameters(velocity_max="2.0")
tracker = MopsTracker(runLocation)
tracker.getDetections(nightly)

------- MOPS Parameters --------
Current Parameter Values:

---- findTracklets ----
	Maximum velocity:                         2.0
	Minimum velocity:                         0.0
---- collapseTracklets ----
	Right Ascension tolerance:                0.002
	Declination tolerance:                    0.002
	Angular tolerance:                        5
	Velocity tolerance:                       0.05
	Method:                                   greedy
	Use RMS filter:                           True
	Maximum RMS:                              0.001
---- purifyTracklets ----
	Maximum RMS:                              0.001
---- removeSubsets (tracklets) ----
	Remove subsets:                           False
	Keep only longest:                        False
---- makeLinkTrackletsInput_byNight.py ----
	Window size:                              15
---- linkTracklets ----
	Detection error threshold:                0.0004
	Maximum right ascension acceleration:     0.02
	Maximum declination acceleration: 

In [25]:
parameters, tracker = runMops.runMops(parameters, tracker, overwrite=True)

------- Run MOPS -------
Running LSST's Moving Object Pipeline

Overwrite triggered: clearing tracker...

Found 4 detection files in /Volumes/DataCenter/neosimData/colin/nightly/.

Saving parameters to /Volumes/DataCenter/neosimData/colin_2016_03_14/parameters.yaml

------- Run MOPS -------
Running findTracklets...
Completed running findTracklets.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_03_14/tracker.yaml

------- Run MOPS -------
Running idsToIndices.py...
Completed running idsToIndices.py.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_03_14/tracker.yaml

------- Run MOPS -------
Running collapseTracklets...
Completed running collapseTracklets.

------- Run MOPS -------
Running indicesToIds.py...
Completed running indicesToIds.py.

Saving tracker to /Volumes/DataCenter/neosimData/colin_2016_03_14/tracker.yaml

------- Run MOPS -------
Running purifyTracklets...
Completed running purifyTracklets.

------- Run MOPS -------
Running indicesToIds.py...
C

In [4]:
parameters = MopsParameters.fromYaml(runLocation + "parameters.yaml")
tracker = MopsTracker.fromYaml(runLocation + "tracker.yaml")

Loading parameters from /Volumes/DataCenter/neosimData/colin_2016_03_14/parameters.yaml
Loading tracker from /Volumes/DataCenter/neosimData/colin_2016_03_14/tracker.yaml


In [5]:
results, df = MopsAnalysis.analyze(parameters, tracker, fullDetFile=dataDrive + "neosimData/colin/dia_sources_to_mops_mod.dat", overwrite=True)

Overwrite triggered: deleting existing results directory...

Initializing new results object...
Creating DiaSources table...
Creating AllObjects table...
Creating FoundObjects view...
Creating MissedObjects view...
Creating AllTracklets table...
Creating TrackletMembers table...
Creating Tracklets view...
Creating CollapsedTracklets view...
Creating PurifiedTracklets view...
Creating FinalTracklets view...

Reading full detections file into dataframe...
Counting findable objects as tracklets...
Counting findable objects as tracks...
Building objects dataframe...
Updating objects dataframe...
Reading full detections file into database...
Starting tracklet analysis for night 56398 at Fri Mar 18 14:59:43 2016

- Writing results to /Volumes/DataCenter/neosimData/colin_2016_03_14/results/56398.results
- Checking file sizes...
- Reading input detections...
- Counting findable objects...
- Updating results object...
Saving results to /Volumes/DataCenter/neosimData/colin_2016_03_14/results/res